In [82]:
import glob
from typing import Generator

import nest_asyncio
import shutil
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import rapidjson
from alive_progress import alive_bar
from keras.optimizer_v2.adam import Adam
from lazyft.data_loader import load_pair_data
from sklearn.preprocessing import StandardScaler

from cnn_model import create_cnn_alternative, create_cnn

from preprocessing import preprocess, create_dataflow_dataframe

import keras
import keras_tuner
import keras_tuner as kt
from keras import layers, losses
from keras.backend import clear_session
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Dense
from keras_preprocessing.image import ImageDataGenerator
from numba import cuda
import util
from data_models import Profile, Mode

nest_asyncio.apply()

# jt -t oceans16 -f roboto -fs 12 -cellw 100%

## Parameters

In [76]:
hp = kt.HyperParameters()
image_size = 224

split = 0
profile = Profile(
    pair="BTC/USDT",
    train_timerange="20210102-20211231",
    test_timerange="20211212-",
    timeframes=["5m", "15m", "30m", "1h"],
    # timeframes=["1h", "4h", "8h", "1d"],
    image_size=image_size,
    batch_size=64,
    # download_interval='15m',
)
profile.ensure_directories_exist()
time_step = 20

## Preprocess

In [ ]:
def zscore(input_data: pd.Series):
    pass


def de_zscore(input_data: pd.Series, standardized_value: float):
    pass

In [83]:
data = load_pair_data(profile.pair, '1h', timerange='20180101-')

2022-04-11 07:55:41.234 | INFO     | lazyft.downloader:download_missing_historical_data:302 - Checking if download is needed for BTC/USDT @ 1h interval(s)
2022-04-11 07:55:41.239 | INFO     | lazyft.downloader:download:371 - Downloading 1561 days worth of market data for BTC/USDT @ 1h ticker-interval(s)...
2022-04-11 07:55:45.904 | INFO     | lazyft.downloader:download_missing_historical_data:334 - Finished downloading data for 1 pairs @ 1h
2022-04-11 07:55:45.905 | INFO     | lazyft.downloader:download_missing_historical_data:339 - Data is up to date
2022-04-11 07:55:45.966 | INFO     | lazyft.data_loader:load_pair_data:40 - Loaded 37474 rows for BTC/USDT, data starts at 2018-01-01 00:00:00+00:00


In [84]:
from sklearn.model_selection import train_test_split
from pandas_ta import zscore, roc

nd = data.copy()
nd['roc'] = roc(nd['close'], length=1)
nd['answer'] = nd['close'].shift(-1)
nd = nd.dropna()
print(nd.head(5).to_markdown())
drop: pd.DataFrame = nd.drop(columns=['date'])

# for k, s in drop.items():
#     s_as_n = s.to_numpy().reshape((1,-1))
#     scaler=  ss.fit(s_as_n)
#     nd.loc[:, k] = scaler.transform(s_as_n)
# gb = data.groupby(np.arange(len(data)) // 10)
gb = util.roll(nd, time_step, sort=False, group_keys=False)
x_input = np.zeros((len(gb), time_step, 6))
y_answers = np.zeros((len(gb), time_step))
with alive_bar(len(gb), force_tty=True) as bar:
    for i, d in gb:
        # for k, s in d.items():
        #     s_as_n = s.to_numpy().reshape((1,-1))
        #     scaler = ss.fit(s_as_n)
        #     nd.loc[:, k] = scaler.transform(s_as_n)
        # print(d.head(10))
        dropped = d.drop(columns=['date']).to_numpy()
        scaler: StandardScaler = StandardScaler().fit(dropped)
        x_scaled = scaler.transform(dropped)
        answers = x_scaled[:, -1]
        x_input[i - 1] = x_scaled[:, :-1]
        # print(x_scaled, '\n', x_scaled.shape)
        y_answers[i - 1] = answers
        bar()

X_train, X_test, y_train, y_test = train_test_split(x_input, y_answers, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

|    | date                      |    open |    high |     low |   close |   volume |       roc |   answer |
|---:|:--------------------------|--------:|--------:|--------:|--------:|---------:|----------:|---------:|
|  1 | 2018-01-01 01:00:00+00:00 | 13529   | 13595.9 | 13155.4 | 13203.1 |  383.697 | -2.40927  |  13330.2 |
|  2 | 2018-01-01 02:00:00+00:00 | 13203   | 13418.4 | 13200   | 13330.2 |  429.065 |  0.962807 |  13410   |
|  3 | 2018-01-01 03:00:00+00:00 | 13330.3 | 13611.3 | 13290   | 13410   |  420.087 |  0.599017 |  13601   |
|  4 | 2018-01-01 04:00:00+00:00 | 13435   | 13623.3 | 13322.1 | 13601   |  340.807 |  1.42416  |  13559   |
|  5 | 2018-01-01 05:00:00+00:00 | 13615.2 | 13699   | 13526.5 | 13559   |  404.229 | -0.308948 |  13780.4 |
|████████████████████████████████████████| 37453/37453 [100%] in 21.8s (1719.05/s)                                      


In [86]:
print(
    f'\nTraining input samples: {len(X_train)}',
    f'Validation input samples: {len(X_val)}',
    f'Testing input samples: {len(X_test)}',
    sep='\n'
)


Training input samples: 22471
Validation input samples: 7491
Testing input samples: 7491


## Build

In [92]:
learning_rate_reduction = ReduceLROnPlateau(
    monitor="val_loss", patience=3, verbose=0, factor=0.5, min_lr=0.00001
)

In [98]:
def build_model(hp: kt.HyperParameters = kt.HyperParameters()):
    kernel_size = hp.Choice('conv1_kernel', values=[1, 3, 5, 7], default=1)
    conv1_filters = hp.Int('conv1_filters', min_value=16, max_value=128, step=16, default=32)
    lstm_units = hp.Int('lstm_units', min_value=32, max_value=256, default=64, step=16)
    model = keras.Sequential()
    input_shape = X_train[0].shape
    model.add(layers.Input(input_shape))
    model.add(layers.Conv1D(conv1_filters, kernel_size=kernel_size, activation='tanh', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=1, padding='same'))
    model.add(layers.LSTM(lstm_units, activation='tanh'))
    model.add(layers.Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    # model.summary(expand_nested=True)
    return model

## Search

In [100]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_loss',
    max_trials=10,
    overwrite=True,
    directory="tuner",
    project_name=profile.datestamp,
    executions_per_trial=1,
    # hyperparameters=hp
)

In [101]:

tuner.search(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=64,
    epochs=10,
    # callbacks=[keras.callbacks.TensorBoard("tuner/tb/" + profile.datestamp)],
)
clear_session()

Trial 10 Complete [00h 00m 29s]
val_loss: 0.7937811613082886

Best val_loss So Far: 0.7937811613082886
Total elapsed time: 00h 04m 22s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = build_model()

In [93]:
model.fit(
    X_train, y_train, epochs=100,
    validation_data=(X_val, y_val),
    batch_size=64,
    callbacks=[learning_rate_reduction],
)
clear_session()

Epoch 1/100
352/352 [==============================] - 3s 7ms/step - loss: 0.7929 - val_loss: 0.7932 - lr: 0.0010
Epoch 2/100
352/352 [==============================] - 2s 4ms/step - loss: 0.7929 - val_loss: 0.7932 - lr: 0.0010
Epoch 3/100
352/352 [==============================] - 1s 4ms/step - loss: 0.7929 - val_loss: 0.7932 - lr: 0.0010
Epoch 4/100
352/352 [==============================] - 1s 4ms/step - loss: 0.7929 - val_loss: 0.7932 - lr: 0.0010
Epoch 5/100
352/352 [==============================] - 1s 4ms/step - loss: 0.7927 - val_loss: 0.7931 - lr: 5.0000e-04
Epoch 6/100
352/352 [==============================] - 1s 4ms/step - loss: 0.7927 - val_loss: 0.7931 - lr: 5.0000e-04
Epoch 7/100
352/352 [==============================] - 1s 4ms/step - loss: 0.7927 - val_loss: 0.7931 - lr: 5.0000e-04
Epoch 8/100
352/352 [==============================] - 2s 4ms/step - loss: 0.7927 - val_loss: 0.7931 - lr: 5.0000e-04
Epoch 9/100
352/352 [==============================] - 2s 5ms/step - los

KeyboardInterrupt: 

### VGG 16
<img src="https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png" width="500" height="300" />

In [ ]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(
    include_top=False,
    weights=None,
    input_shape=test_img[0].shape,
    classifier_activation='softmax'
)

In [11]:
model.evaluate(validate_generator)
clear_session()

15/15 [==============================] - 1s 88ms/step - loss: 0.1240 - accuracy: 0.9641


[0.1239885613322258, 0.9640591740608215]

In [28]:
model.predict(test_generator)
clear_session()

array([[1.73703267e-03, 9.98262942e-01],
       [1.65773654e-05, 9.99983430e-01],
       [6.44575746e-04, 9.99355376e-01],
       [9.99999523e-01, 5.23124640e-07],
       [9.99993682e-01, 6.29351462e-06],
       [9.99997139e-01, 2.81512689e-06],
       [1.88813135e-02, 9.81118679e-01],
       [3.22594610e-03, 9.96774018e-01],
       [9.97419238e-01, 2.58078892e-03],
       [8.33522121e-04, 9.99166489e-01],
       [9.53888595e-01, 4.61113565e-02],
       [1.20793134e-02, 9.87920642e-01],
       [4.00322629e-03, 9.95996833e-01],
       [3.96189193e-04, 9.99603808e-01],
       [3.31265875e-03, 9.96687353e-01],
       [1.51765504e-04, 9.99848247e-01],
       [2.36065313e-03, 9.97639418e-01],
       [2.10060249e-03, 9.97899413e-01],
       [9.99999046e-01, 9.70398446e-07],
       [9.99909043e-01, 9.09386072e-05],
       [9.99989748e-01, 1.02635622e-05],
       [1.69562735e-02, 9.83043730e-01],
       [9.99999285e-01, 7.63856178e-07],
       [9.85830963e-01, 1.41690485e-02],
       [9.999997

In [12]:
model.save("90pcta")
profile.save(Path("90pcta"))

2022-04-07 16:35:13.063686: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 90pcta/assets
2022-04-07 16:35:13.612 | INFO     | data_models:save:115 - Saved profile to 90pcta/profile.json


## Summarize

In [11]:
# print(tuner.results_summary())
print(tuner.get_best_hyperparameters()[0].values)

{'conv1_units': 32, 'conv1_kernel': 3, 'conv2_units': 112, 'conv2_kernel': 3, 'conv3_units': 128, 'conv3_kernel': 3, 'conv4_units': 96, 'conv4_kernel': 5, 'conv5_units': 128, 'conv5_kernel': 5, 'conv6_units': 32, 'conv6_kernel': 5, 'conv7_units': 32, 'conv7_kernel': 3, 'dropout1': 0.0, 'dropout2': 0.5, 'loss': 'mean_absolute_error'}


## Evaluate

In [30]:
best_model = tuner.get_best_models()[0]
scores = best_model.evaluate(test_generator, return_dict=True)

88/88 [==============================] - 8s 92ms/step - loss: 0.7006 - accuracy: 0.5434


In [31]:
if scores["accuracy"] > 0.54:
    best_model.save(Path("tuner", profile.datestamp, "best_model.h5"))
    profile.save(directory=Path("tuner", profile.datestamp))
    best_model.summary()
else:
    shutil.rmtree(Path("tuner", profile.datestamp))
clear_session()

2022-04-08 19:14:10.606 | INFO     | data_models:save:117 - Saved profile to tuner/20220408181048/profile.json
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 36, 36, 32)        2432      
                                                                 
 batch_normalization (BatchN  (None, 36, 36, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 256)       205056    
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 256)      1024      
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 32)        73760     
           

In [22]:
model = util.load_specific_tuner_model("tuner/20220407120059")[0][1]
model.evaluate(test_generator, return_dict=True)

2022-04-07 12:43:16.752140: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302


1/1 [==============================] - 1s 809ms/step - loss: 0.4992 - accuracy: 0.6250


2022-04-07 12:43:17.019377: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-07 12:43:17.019846: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-07 12:43:17.019863: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-07 12:43:17.020327: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-07 12:43:17.020355: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


{'loss': 0.49920588731765747, 'accuracy': 0.625}

## Backtest

In [3]:
# del best_model
from lazyft.command_parameters import BacktestParameters

params = BacktestParameters(
    config_path="user_data/config.json",
    # timerange='20211121-20211212',
    # timerange='20210110-20210131',
    # timerange='20220102-20220116',
    timerange="20220101-",
    interval=profile.min_tf,
    # days=365,
    stake_amount="unlimited",
    starting_balance=1000,
    max_open_trades=1,
    pairs=[
        "BTC/USDT",
        # 'ETH/USDT',
    ],
    download_data=True,
    custom_settings={
        "min_buy_threshold": 0.5,
        "specific_model":    "tuner/20220408181048",
        #         "timeframes": profile.timeframes,
    },
)

In [4]:
runner = params.run("SagesGymCNN2", load_from_hash=False, stdout=True)
report = runner.save()
print(
    report.trades[
        ["pair", "open_date", "close_date", "open_rate", "close_rate", "profit_abs"]
    ]
)

2022-04-09 02:28:35.754 | INFO     | lazyft.parameter_tools:remove_params_file:36 - Removing strategy params: /home/raphael/PycharmProjects/cnn_gaf/user_data/strategies/my_gym_cnn2.json
2022-04-09 02:28:35.784 | INFO     | lazyft.strategy:save_strategy_text_to_database:188 - Strategy SagesGymCNN2 already in database...skipping
2022-04-09 02:28:35.785 | INFO     | lazyft.backtest.runner:update_spaces:384 - Updating custom spaces...
2022-04-09 02:28:36.913 | INFO     | lazyft.downloader:download_missing_historical_data:302 - Checking if download is needed for BTC/USDT, BTC/USDT @ 5m interval(s)
2022-04-09 02:28:36.916 | INFO     | lazyft.downloader:download_missing_historical_data:339 - Data is up to date
2022-04-09 02:28:37.999 | INFO     | lazyft.backtest.runner:pre_execute:179 - Running command: "freqtrade backtesting -c /home/raphael/PycharmProjects/cnn_gaf/user_data/config.json --user-data-dir /home/raphael/PycharmProjects/cnn_gaf/user_data --strategy-path /home/raphael/PycharmProje

2022-04-09 02:28:38.863641: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:28:38.885467: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:28:38.885580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-09 02:28:38.885933: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

2022-04-09 02:28:42.935 | INFO     | writer:fill_writer_queue:61 - Creating writer queue... | Save folder: /tmp/BTC_USDT/1640707200-1649455200/5m_15m_30m_1h
Filename: /home/raphael/PycharmProjects/cnn_gaf/preprocessing.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    90  1383.40 MiB  1383.40 MiB           1   @memory_profiler.profile(precision=2)
    91                                         def preprocess_prediction(custom_timeframes, df, profile):
    92                                             """
    93                                             It takes a dataframe, splits it into timeframes, writes the images to disk, and returns the path to
    94                                             the folder where the images are stored
    95                                         
    96                                             :param custom_timeframes: a list of timeframes to use for prediction. If you want to use all the
    97                          

2022-04-09 02:28:44.233430: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-04-09 02:28:44.548863: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-09 02:28:44.549243: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-09 02:28:44.549256: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-09 02:28:44.549599: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-09 02:28:44.549626: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


202112291200 -> 0 -> 0
202112291300 -> 1 -> 1
202112291400 -> 1 -> 1
202112291500 -> 1 -> 1
202112291600 -> 1 -> 1
202112291700 -> 0 -> 0
202112291800 -> 0 -> 0
202112291900 -> 1 -> 1
202112292000 -> 1 -> 1
202112292100 -> 1 -> 1
202112292200 -> 1 -> 1
202112292300 -> 1 -> 1
202112300000 -> 0 -> 0
202112300100 -> 1 -> 1
202112300200 -> 1 -> 1
202112300300 -> 0 -> 0
202112300400 -> 1 -> 1
202112300500 -> 1 -> 1
202112300600 -> 1 -> 1
202112300700 -> 0 -> 0
202112300800 -> 0 -> 0
202112300900 -> 1 -> 1
202112301000 -> 0 -> 0
202112301100 -> 1 -> 1
202112301200 -> 1 -> 1
202112301300 -> 1 -> 1
202112301400 -> 0 -> 0
202112301500 -> 0 -> 0
202112301600 -> 0 -> 0
202112301700 -> 1 -> 1
202112301800 -> 1 -> 1
202112301900 -> 1 -> 1
202112302000 -> 1 -> 1
202112302100 -> 1 -> 1
202112302200 -> 1 -> 1
202112302300 -> 1 -> 1
202112310000 -> 1 -> 1
202112310100 -> 1 -> 1
202112310200 -> 0 -> 0
202112310300 -> 1 -> 1
202112310400 -> 0 -> 0
202112310500 -> 1 -> 1
202112310600 -> 1 -> 1
20211231070

In [21]:
from lazyft.reports import get_backtest_repo

report = get_backtest_repo().first()

In [5]:
# print(get_backtest_repo().tail(3))
# print(report.backtest_data['trades'])
print(report.trades[['open_date', 'close_date', 'close_rate', 'open_rate', 'profit_ratio']].to_markdown())

|     | open_date                 | close_date                |   close_rate |   open_rate |   profit_ratio |
|----:|:--------------------------|:--------------------------|-------------:|------------:|---------------:|
|   0 | 2022-01-01 00:55:00+00:00 | 2022-01-01 01:55:00+00:00 |      46769.6 |     46690.2 |    -0.00029976 |
|   1 | 2022-01-01 05:55:00+00:00 | 2022-01-01 06:55:00+00:00 |      47072.3 |     47240.7 |    -0.00555518 |
|   2 | 2022-01-01 07:55:00+00:00 | 2022-01-01 09:00:00+00:00 |      47124.6 |     47094.7 |    -0.00136311 |
|   3 | 2022-01-01 09:55:00+00:00 | 2022-01-01 10:55:00+00:00 |      46938   |     47068.5 |    -0.00476586 |
|   4 | 2022-01-01 11:55:00+00:00 | 2022-01-01 12:55:00+00:00 |      47100   |     46851.7 |     0.00329068 |
|   5 | 2022-01-01 13:55:00+00:00 | 2022-01-01 14:55:00+00:00 |      46895.4 |     47011.6 |    -0.00446501 |
|   6 | 2022-01-01 15:55:00+00:00 | 2022-01-01 17:00:00+00:00 |      47269.2 |     47195.3 |    -0.0004353  |
|   7 | 20

In [10]:
print(report.report_text)

Result for strategy SagesGymCNN2 #14 | Hyperopt: False | Hyperopt id: 
========================================================== BACKTESTING REPORT =========================================================
|     Pair |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |
|----------+--------+----------------+----------------+-------------------+----------------+----------------+-------------------------|
| BTC/USDT |    748 |          -0.28 |        -212.58 |          -881.062 |         -88.11 |        1:01:00 |   241     0   507  32.2 |
|    TOTAL |    748 |          -0.28 |        -212.58 |          -881.062 |         -88.11 |        1:01:00 |   241     0   507  32.2 |
========================================================== BUY TAG STATS ===========================================================
|   TAG |   Buys |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Dr

In [8]:
from pprint import pprint
from keras.models import load_model

model = load_model('tuner/20220408181048/best_model.h5')

2022-04-08 22:42:54.290162: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 22:42:54.312673: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 22:42:54.312797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-08 22:42:54.313404: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [9]:
predict = model.predict(test_generator)
clear_session()

2022-04-08 22:43:01.559525: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8302
2022-04-08 22:43:01.832578: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-08 22:43:01.833041: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-08 22:43:01.833062: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-04-08 22:43:01.833546: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-04-08 22:43:01.833640: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-04-08 22:43:02.330453: W tensorflow/core/common_runtime/bf

[('SHORT', array([0.383921, 0.616079], dtype=float32)),
 ('LONG', array([0.37017214, 0.62982786], dtype=float32))]


In [41]:
print(predict)

[[0.383921   0.616079  ]
 [0.37017214 0.62982786]
 [0.37109217 0.62890786]
 ...
 [0.46331537 0.53668463]
 [0.438827   0.56117296]
 [0.43346682 0.56653315]]


In [38]:
dt = report.trades['open_date'].dt.strftime("%Y%m%d%H%M%S")


In [36]:
help(dt)


Help on DatetimeProperties in module pandas.core.indexes.accessors object:

class DatetimeProperties(Properties)
 |  DatetimeProperties(data: 'Series', orig)
 |  
 |  Accessor object for datetimelike properties of the Series values.
 |  
 |  Examples
 |  --------
 |  >>> seconds_series = pd.Series(pd.date_range("2000-01-01", periods=3, freq="s"))
 |  >>> seconds_series
 |  0   2000-01-01 00:00:00
 |  1   2000-01-01 00:00:01
 |  2   2000-01-01 00:00:02
 |  dtype: datetime64[ns]
 |  >>> seconds_series.dt.second
 |  0    0
 |  1    1
 |  2    2
 |  dtype: int64
 |  
 |  >>> hours_series = pd.Series(pd.date_range("2000-01-01", periods=3, freq="h"))
 |  >>> hours_series
 |  0   2000-01-01 00:00:00
 |  1   2000-01-01 01:00:00
 |  2   2000-01-01 02:00:00
 |  dtype: datetime64[ns]
 |  >>> hours_series.dt.hour
 |  0    0
 |  1    1
 |  2    2
 |  dtype: int64
 |  
 |  >>> quarters_series = pd.Series(pd.date_range("2000-01-01", periods=3, freq="q"))
 |  >>> quarters_series
 |  0   2000-03-31
 | 

In [6]:
import sys
from writer import figure_cache

sys.getsizeof(figure_cache)


48